In [1]:
using Polyhedra

In [2]:
h = SimpleHRepresentation(eye(2), zeros(2))

H-representation
begin
 2 3 real
 0.0 -1.0 -0.0
 0.0 -0.0 -1.0
end

In [3]:
supertype(typeof(h))

Polyhedra.HRepresentation{2,Float64}

In [4]:
typeof(h)

Polyhedra.SimpleHRepresentation{2,Float64}

In [5]:
using StaticArrays

In [7]:
struct Foo{T}
    position::SVector{2, T}
    velocity::SVector{2, T}
end

In [8]:
data = [1, 2, 3, 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [9]:
reinterpret(Foo{Int}, data)

1-element Array{Foo{Int64},1}:
 Foo{Int64}([1, 2], [3, 4])

In [10]:
using JuMP

In [11]:
m = Model()
@variable m x[1:4]

4-element Array{JuMP.Variable,1}:
 x[1]
 x[2]
 x[3]
 x[4]

In [14]:
reinterpret(Ref{Foo{eltype(x)}}, x)

LoadError: [91margument is an abstract type; size is indeterminate[39m

In [4]:
module Box

using Parameters
using Polyhedra
using StaticArrays
import Base: convert

function from_bounds(lb::AbstractVector, ub::AbstractVector)
    len = length(lb)
    @assert length(lb) == length(ub)
    SimpleHRepresentation(vcat(eye(len); .-eye(len)), vcat(lb, .-ub))
end

@enum Body CoM LeftLeg RightLeg LeftHand RightHand

@with_kw struct BoxAtlas{T}
    position_limits::Dict{Body, SimpleHRepresentation{2, T}} = Dict(
        CoM=>from_bounds([0.3, 0.3], [0.7, 0.7]),
        LeftFoot=>from_bounds([0.0, -0.7], [0.4, -0.3]),
        RightFoot=>from_bounds([-0.4, -0.7], [0.0, -0.3]),
        LeftHand=>from_bounds([0.2, -0.1], [0.6, 0.3]),
        RightHand=>from_bounds([-0.6, -0.1], [-0.2, 0.3])
    )
    velocity_limits::Dict{Body, SimpleHRepresentation{2, T}} = Dict(
        CoM=>from_bounds([-5., -5], [5, 5]),
        LeftFoot=>from_bounds([-5., -5], [5, 5]),
        RightFoot=>from_bounds([-5., -5], [5, 5]),
        LeftHand=>from_bounds([-5., -5], [5, 5]),
        RightHand=>from_bounds([-5., -5], [5, 5]),
        )
    mass::T = 1.0
    stiffness::T = 100.
    gravity::T = 10.
    Δt::T = 0.1
end

struct State{T}
    position::Dict{Body, SVector{2, T}}
    velocity::Dict{Body, SVector{2, T}}
end

convert(::Type{State}, x::AbstractVector{T}) where {T} = convert(State{T}, x)

function convert(::Type{State{T}}, x::AbstractVector{T}) where {T}
    State{T}(
        Dict(
            CoM => x[1:2], 
            LeftLeg => x[3:4],
            RightLeg => x[5:6],
            LeftHand => x[7:8],
            RightHand => x[9:10]
            ),
        Dict(
            CoM => x[11:12], 
            LeftLeg => x[13:14],
            RightLeg => x[15:16],
            LeftHand => x[17:18],
            RightHand => x[19:20]
            )
        )
end

end
    

Box